# GPT原理精讲与架构复现

### 序1 最强GPT架构究竟是哪一代架构？

> **从Transformer到GPT**

从 Transformer 到 GPT 系列模型的发展经历了以下关键改进：
1. 从双向编码器-解码器架构转变为**自回归解码器**，专注于文本生成任务。
2. 通过**轴向位置编码**和**缓存机制**应对上下文遗忘问题。
3. 引入了 **Macaron Attention**、增量推理、梯度检查点等结构和技术，优化模型的生成质量和推理速度。
4. 在训练过程中应用 **梯度裁剪**、辅助损失、混合精度和分布式训练，加速模型收敛并节省资源。

这些改进使得 GPT 系列在规模和性能上逐步突破，成为现代 NLP 任务中的主流模型之一。

> 从 **GPT-2** 到 **GPT-3**：规模的演变

- **模型规模的提升**：GPT-3 显著增加了参数数量，扩展到 **1750亿参数**，这是 GPT-2 参数量（15亿）的大约 100 倍。这种大规模参数量的模型极大地增强了 GPT-3 的语言生成能力，使其能够更好地理解和生成复杂文本。

- **训练数据**：GPT-3 采用了大量高质量的公开文本数据进行训练，覆盖了互联网、书籍和维基百科等多种来源。广泛的数据提高了模型对通用知识和多种语言的理解能力。

- **主要应用**：GPT-3 的一个核心创新是 Few-Shot 学习能力，即在不需要大量任务特定训练的情况下，通过在提示（Prompt）中给出少量的示例，模型就能理解并完成类似任务。这得益于 GPT-3 在超大规模数据集上进行的预训练，使得它能够在多种任务中展现出良好的表现。GPT-3 以其生成、总结、翻译等语言处理任务的卓越能力在 NLP 社区引起了广泛关注，但由于缺乏对用户指令的特定优化，GPT-3 的输出在一些应用场景中存在准确性和安全性问题。

> **GPT-3** 到 **GPT-3.5**：RLHF是核心

**GPT-3.5**，也称 **InstructGPT** 或 **ChatGPT**，是在 GPT-3 基础上的一个重大改进，特别是在用户指令的响应和控制方面的提升。InstructGPT 通过引入 **RLHF（Reinforcement Learning from Human Feedback）** 来优化模型的行为，使得它更适合与用户交互。

InstructGPT 的最大创新是通过 **人类反馈的强化学习**（RLHF）来优化模型的生成。具体来说，先用少量的指令数据对模型进行 **监督微调（Supervised Fine-Tuning, SFT）**，然后通过训练一个 **奖励模型（Reward Model）** 评估生成的质量，最后使用强化学习优化模型，使其生成符合人类偏好的内容。具体过程是，首先收集人类标注的指令-响应数据，训练一个初始的 SFT 模型；然后，人类标注员为生成的多种输出打分，用这些评分训练一个奖励模型；最后，使用奖励模型的反馈来微调模型参数，使得模型更符合人类的指令要求。
  

> 从 **GPT-3.5** 到 **GPT-4**：更大、更多模态、更长的上下文、可能存在CoT

- **多模态能力**：GPT-4 引入了 **多模态**（Multimodal）输入能力，不仅可以处理文本，还可以处理图像。这意味着 GPT-4 不仅能够对文字进行理解和生成，还能够理解图像内容（例如图像描述、图像中的物体识别等）。不过，具体的多模态能力在不同版本的 GPT-4 中可能有不同的实现方式。

- **更大的模型规模**：虽然 OpenAI 并未公开 GPT-4 的具体参数数量，但通常认为 GPT-4 比 GPT-3.5 的参数量更大。通过增加参数和改进训练方法，GPT-4 的生成质量和理解能力都得到了显著提升。

- **更强的逻辑推理和一致性**：GPT-4 对于复杂的推理任务和对话一致性有了明显提升。例如，GPT-4 在法律、医学、数学等需要逻辑推理的领域表现更好。GPT-4 能够更好地保持对话上下文，并在连续对话中更一致地回应用户。

| 模型       | 参数量               | 核心进步                  | 主要技术         | 典型应用                      |
|------------|----------------------|---------------------------|------------------|-------------------------------|
| **GPT-2**  | 15亿                 | 更大的文本理解能力        | Transformer     | 文本生成、翻译                 |
| **GPT-3**  | 1750亿               | Few-Shot 学习             | 扩大数据规模     | 高级文本生成、问答            |
| **GPT-3.5**（InstructGPT）| 1750亿左右           | 基于人类反馈的优化        | RLHF             | ChatGPT、指令理解             |
| **GPT-4**  | 未公开（可能更大）    | 多模态能力和逻辑推理       | 多模态训练、RLHF | 高级对话、多模态任务           |

### 序2 GPT3架构一览

<center><img src="https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/17.png" alt="描述文字" width="300">

In [58]:
import math
import struct
import inspect
import time

import LMConfig
from typing import Any, Optional, Tuple
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from transformers import PreTrainedModel
from transformers.modeling_outputs import CausalLMOutputWithPast

## 1. Embedding层与轴向位置编码

### 1.1 带Dropout的Embedding层

在PyTorch中，`nn.Embedding`层是用于处理离散数据（如单词或类别）的关键组件，特别常见于自然语言处理（NLP）和推荐系统等任务。它的主要功能是将输入的整数索引映射到连续的高维向量空间中，即将**索引**转化为**嵌入向量**。

```python
torch.nn.Embedding(num_embeddings, embedding_dim)
torch.nn.Dropout(p)
```

- **`num_embeddings`**: 嵌入表的大小，即词汇表的大小或类别数。它定义了有多少个不同的“离散输入”可以映射到嵌入向量。
- **`embedding_dim`**: 每个离散输入（类别、单词等）将被映射到的连续向量的维度大小，也就是常说的d_model。

`nn.Embedding`的输入通常是整数（类别索引或词汇索引），它会根据输入的索引从一个大小为 `(num_embeddings, embedding_dim)` 的查找表中检索出相应的嵌入向量。

In [12]:
import torch
import torch.nn as nn

# 定义Embedding层
embedding = nn.Embedding(10, 3)  # num_embeddings=10, d_model=3
dropout = nn.Dropout(0.1)

# 输入索引
input_indices = torch.tensor([1, 2, 3])

# 获取嵌入向量
output = dropout(embedding(input_indices))

print(output)

tensor([[-0.1612, -0.0894,  0.0000],
        [ 0.1517, -1.3646, -0.7361],
        [ 0.0000,  0.5655,  0.2440]], grad_fn=<MulBackward0>)


- embedding层携带巨大的权重矩阵，是参数量计算的关键过程之一

In [14]:
print(embedding.weight) #结构为10,3

Parameter containing:
tensor([[ 0.4752, -0.2457,  0.2101],
        [ 1.5522,  0.7179,  1.6805],
        [ 2.1118,  0.2995,  0.4167],
        [-0.6033, -0.4972, -1.6700],
        [-0.8719, -0.7207,  0.8305],
        [ 1.2962, -1.2880,  0.8838],
        [-0.7804, -0.1872,  0.3502],
        [-0.2817, -0.9322,  0.5499],
        [-0.5277,  0.8808, -1.6055],
        [ 0.5706,  0.9455, -0.0734]], requires_grad=True)


### 1.2 轴向位置编码Axial Positional Encoding

在自然语言处理或图像处理等任务中，传统的 Transformer 使用 **绝对位置编码**，即为每个位置赋予一个唯一的向量表示，直接用来表示序列中每个位置的信息。虽然这种编码方式在一般的 NLP 任务中表现良好，但在更长的序列（例如处理长文档、视频帧序列、或图片像素）中效率较低，因为每个位置都需要一个独立的编码。**轴向位置编码（Axial Positional Encoding）** 是一种用于表示序列中位置的编码方式，尤其适用于高维度或长序列的场景。相比于传统的绝对位置编码，轴向位置编码能够更加高效地表示复杂序列中的位置关系。
**轴向位置编码** 的核心在于 **多轴（Multiple Axes）** 位置编码的技术。比如，在图像或长序列任务中，可以把二维或更高维的序列分解成多个轴向，每个轴向分别编码一个维度的位置。这样能更高效且更节约内存地表示长序列中的位置信息。

- **轴向位置编码的具体流程**

假设现在我们有一个文字序列，有一个主要的**位置轴**（即 `seq_len`），我们可以将这个单一的轴虚拟拆分为两个子轴。具体来说，我们要将 **`seq_len` 拆分成两个更小的维度**，然后分别对这两个维度进行位置编码，最后再合并这些编码来表示整个序列。

1. **定义两个轴向维度**：
   - 定义两个参数 `axial_dim_1` 和 `axial_dim_2` ，并假设 `seq_len = axial_dim_1 * axial_dim_2`，我们可以将序列视为一个虚拟的 2D 结构，大小为 `(axial_dim_1, axial_dim_2)`。
   - 例如，如果 `seq_len = 128`，可以拆分成 `axial_dim_1 = 16` 和 `axial_dim_2 = 8`，得到一个 16x8 的虚拟网格。
<br><br>
2. **创建两个轴向维度上的Embedding矩阵**：
   - 使用 `axial_wpe_1` 和 `axial_wpe_2` 表示两个轴的编码，基于这两个编码创建嵌入矩阵
     ```python
        axial_wpe_1 = nn.Parameter(torch.randn(axial_dim_1, d_model) * 0.01)
        axial_wpe_2 = nn.Parameter(torch.randn(axial_dim_2, d_model) * 0.01)
     ```
   - 其中，`axial_wpe_1` 和 `axial_wpe_2` 是分别在 `axial_dim_1` 和 `axial_dim_2` 维度上的嵌入矩阵，每个矩阵的大小分别为 `(axial_dim_1, d_model)` 和 `(axial_dim_2, d_model)`。
<br><br>
3. **扩展和广播编码**：
   - 通过 `unsqueeze` 和 `expand` 操作将 `axial_wpe_1` 和 `axial_wpe_2` 广播为相同的形状 `(axial_dim_1, axial_dim_2, d_model)`。
   - 这样，我们得到了两个包含二维位置编码的矩阵，形状为 `(axial_dim_1, axial_dim_2, d_model)`。
<br><br>
4. **组合轴向编码并 reshape**：
   - 将广播后的 `axial_wpe_1` 和 `axial_wpe_2` **相加并取平均**，形成一个位置编码矩阵 `wpe`，即 `(axial_wpe_1 + axial_wpe_2) / 2`。
   - 最后，将这个矩阵调整形状（reshape）为 `[seq_len, d_model]`，这样整个位置编码可以用于序列输入。

- **轴向位置编码的优势**

> - **降低内存消耗、同时减少计算量**：相比为每个位置生成独立的编码向量，轴向位置编码通过将高维度分解为多个轴向显著降低了内存需求。举例说明——

假设我们有 **128 个位置（seq_len = 128）**，每个位置的嵌入维度为 **64（embedding_dim = 64）**。

1. **普通位置编码**：
   - 每个位置需要一个独立的嵌入向量，因此需要一个形状为 `(128, 64)` 的位置编码矩阵。
   - 总参数量为：`128 * 64 = 8192`。

2. **轴向位置编码**：
   - 将 128 个位置分解成两个轴向，比如 **`axial_dim_1 = 16` 和 `axial_dim_2 = 8`**。这样，我们可以将 128 个位置表示为一个 **16x8 的二维网格**。
   - 对于 `axial_dim_1` 和 `axial_dim_2`，分别创建两个嵌入矩阵 `axial_wpe_1` 和 `axial_wpe_2`：
     - `axial_wpe_1` 的形状为 `(16, 64)`。
     - `axial_wpe_2` 的形状为 `(8, 64)`。
   - 总参数量为：`16 * 64 + 8 * 64 = 1536 + 512 = 2048`。

- **普通位置编码**的参数总量：8192。
- **轴向位置编码**的参数总量：2048（减少了 75% 的参数量）。

这就意味着，轴向位置编码显著减少了存储每个位置编码所需的内存。这种方法在长序列任务中尤为重要，因为位置编码的内存需求随序列长度线性增加，而轴向编码的参数量只随轴向的长度增加，内存需求增长缓慢。并且，在计算时，轴向位置编码可以利用两个轴的编码矩阵进行广播和相加，而不需要为每个位置分别计算位置编码，**减少了计算量**。将 `axial_wpe_1` 和 `axial_wpe_2` 广播到相同的 `(16, 8, 64)` 形状，再相加即可得到完整的 128 个位置的编码。广播操作相比逐一生成位置编码更加高效，特别是在大型矩阵操作时。

---

> - **更好地处理长序列或高维数据**：在长序列任务中，轴向位置编码能够更好地扩展并捕捉到数据中的位置信息。

在长序列任务中，**轴向位置编码**能够更好地扩展并捕捉数据中的位置信息，主要原因在于它的**分解式编码方式**和**更细粒度的位置信息捕捉**。这里从以下几个方面来解释：

首先，轴向位置编码将一个长序列分解为多个轴向（例如 2 个轴向），分别为每个轴提供一个位置编码。这种分解可以理解为对**二维网格的编码**，相比普通位置编码的单一轴向编码，提供了更多维度的信息。具体来说：

- **每个轴提供独立的位置信息**：在分解成 `axial_dim_1` 和 `axial_dim_2` 之后，每个轴都提供了一组独立的编码。这意味着在一个轴向位置上能够捕捉到更多细粒度的信息。
- **丰富的组合信息**：每个位置的编码是两个轴向编码的组合，这使得位置编码可以利用两个轴向的不同信息来编码不同的位置关系。这一点在图像信息上表现得更明显，对图像而言一个 `16x8` 的二维网格编码会比单一的 128 个位置的编码提供更丰富的位置信息组合。对文字数据来说，优势更多在“两组参数同时控制一个位置编码、相当于实现对文字的两种解释”上。

除此之外，将长序列拆解为多个轴向，使模型可以更灵活地处理局部和全局的信息，尤其在高维序列（如图像、长文本）中尤为有效：

- **局部捕捉**：轴向编码在每个轴中包含较小的跨度（如 `axial_dim_1=16` 表示行方向、`axial_dim_2=8` 表示列方向），因此每个轴向编码能够专注于局部区域的关系。这对于捕捉局部依赖性（如句子中的词组、图像中的小区域）非常有用。
- **全局整合**：通过将两个轴向编码的组合应用于整体，能够在更长范围内建立全局信息的关联，使得模型可以在更广的上下文中找到各位置的相对关系。
- **减少位置偏移的影响**：在长序列的普通位置编码中，编码值随位置的变化较大，而轴向编码的分解方式能使得位置偏移在局部范围内得到缓解，从而减少模型对绝对位置的过度依赖。

同时别忘了，轴向位置编码不仅在表示上能够更灵活，在计算效率和内存需求上也更适合处理长序列。由于只需存储轴向的编码矩阵而非完整序列长度编码，因此当序列长度进一步增加时，轴向编码的存储增长较慢。这种优势使得它比普通位置编码更能扩展到长序列任务中。

- **轴向位置编码层的实现**

In [15]:
import torch
import torch.nn as nn

#定义轴向位置编码层
def axial_positional_emb(embedding_dim, axial_dim_1, axial_dim_2):
    # 1. 创建每个轴向的嵌入矩阵
    axial_wpe_1 = nn.Parameter(torch.randn(axial_dim_1, embedding_dim) * 0.01)
    axial_wpe_2 = nn.Parameter(torch.randn(axial_dim_2, embedding_dim) * 0.01)

    # 2. 广播轴向嵌入到相同形状
    axial_wpe_1 = axial_wpe_1.unsqueeze(1).expand(-1, axial_dim_2, embedding_dim)
    axial_wpe_2 = axial_wpe_2.unsqueeze(0).expand(axial_dim_1, -1, embedding_dim)

    # 3. 组合轴向位置编码
    wpe = (axial_wpe_1 + axial_wpe_2) / 2

    # 4. 调整形状并返回
    wpe = wpe.view(axial_dim_1 * axial_dim_2, embedding_dim)
    return wpe

In [16]:
# 应用建好的WPE结果

# 假设输入数据
batch_size = 32
seq_len = 128
embedding_dim = 64
axial_dim_1, axial_dim_2 = 16, 8  # 确保 axial_dim_1 * axial_dim_2 == seq_len

# 生成位置编码
wpe = axial_positional_emb(embedding_dim, axial_dim_1, axial_dim_2)

# 假设嵌入输入数据的形状为 (batch_size, seq_len, embedding_dim)
input_embeddings = torch.randn(batch_size, seq_len, embedding_dim)

# 将 wpe 扩展到批次维度
wpe = wpe.unsqueeze(0).expand(batch_size, -1, -1)

# 将位置编码与输入嵌入相加
input_with_positional_encoding = input_embeddings + wpe

In [18]:
input_with_positional_encoding.shape

torch.Size([32, 128, 64])

## 2. 前馈神经网络FFN with GLU

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/18.png)

左边的图像展示了经典 Transformer 中的前馈神经网络结构。输入经过一个线性层将维度从 `d_model` 映射到更高的隐藏维度 `hidden_dim`，然后通过 ReLU 激活，再经过另一个线性层将维度映射回 `d_model`，最后通过 Dropout 输出。这种设计为输入数据提供了非线性变换和基本的正则化。

右边的图像展示了 GPT 模型中的前馈神经网络结构，采用了门控线性单元（GLU）。输入首先通过一个线性层映射到 `2 * hidden_dim`，然后分成两部分，其中一部分经过 sigmoid 激活作为门控信号，与另一部分相乘形成门控后的输出。接着通过另一个线性层映射回 `d_model` 维度并输出。GLU 的设计使网络能够选择性地传递信息，提升模型的灵活性和表达能力。

**门控线性单元（Gated Linear Unit，GLU）** 是一种通过门控机制控制信息流的神经网络单元。在 GLU 中，输入被分成两个部分：一部分作为主输出，另一部分作为门控。通过对门控部分进行激活，然后与主输出相乘，GLU 能够控制信息流动，从而提高网络的表示能力和选择性。

假设输入向量 $ x $ 的维度为 $ d $，则 GLU 会将输入维度扩展为 $ 2d $ 以便分为两部分。GLU 的公式如下：
$$
\text{GLU}(x) = \left( x_1 \right) \times \sigma(x_2)
$$

其中：

- $ x_1 $ 和 $ x_2 $ 是输入向量 $ x $ 的两个分量，分别对应主输出和门控部分，维度均为 $ d $。
- $ \sigma(x_2) $ 是对 $ x_2 $ 应用 **Sigmoid 激活函数**，产生一个范围在 $[0, 1]$ 的门控值，用来控制信息的流动。
- $ x_1 \times \sigma(x_2) $ 表示按元素相乘，产生 GLU 的输出。

在代码实现中，通常会通过一个线性层将输入映射到 `2 * d` 维度，之后再将输出拆分为两部分分别作为 $ x_1 $ 和 $ x_2 $。

- **GLU 的作用**

1. **信息控制**：GLU 通过 Sigmoid 门控，使网络能够选择性地传递信息。Sigmoid 激活会使某些信息通过门控输出，而另一些信息被抑制或完全阻止。
2. **提高模型的非线性表达能力**：相比于普通的线性单元，GLU 能引入更丰富的非线性关系，提升模型的表现力。
3. **降低计算复杂度**：GLU 的结构简单，仅在主输出上应用门控操作，计算开销低，适合在深层网络中使用。

GLU 在自然语言处理、推荐系统和其他深度学习任务中较为常用，能够在不显著增加计算负担的情况下提升模型性能。

- **带GLU的门控前馈神经网络的实现**

In [19]:
import torch
import torch.nn as nn

class MLP_GLU(nn.Module):
    def __init__(self, dim: int, hidden_dim: int, dropout: float):
        super().__init__()
        
        # 定义线性层，w1 实现线性投影到 2 * hidden_dim，用于 GLU 分裂
        self.w1 = nn.Linear(dim, 2 * hidden_dim, bias=False)
        self.w2 = nn.Linear(hidden_dim, dim, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # 1. 第一个线性变换，将 x 映射到 2 * hidden_dim 维度
        h = self.w1(x)

        # 2. 将 h 分成两部分，一半作为 h 主通道，另一半作为 gate 门控
        h, gate = h.chunk(2, dim=-1)

        # 3. 使用 sigmoid 激活 gate，形成 GLU 门控
        h = h * torch.sigmoid(gate)

        # 4. 使用第二个线性层将 h 投影回 dim
        h2 = self.w2(h)

        # 5. 应用 dropout 并返回
        return self.dropout(h2)

## 3. GPT中的Macaron注意力机制

<center><img src="https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/17.png" alt="描述文字" width="300">

**Macaron 注意力机制**是一种前馈网络和注意力机制的改进结构，主要应用于长序列建模任务。它最初由 Microsoft 研究团队在“Macaron Net: Integrating Feedforward and Recurrent Networks for Long-Term Sequence Modeling”一文中提出。Macaron 结构被证明在自然语言处理、语音识别等任务上表现出色。相比于传统的 Transformer 结构，Macaron 引入了双前馈网络来增强模型的表示能力。

在经典 Transformer 中，每个 Transformer 层由一个前馈网络和一个多头注意力机制组成。而 Macaron 注意力机制的主要改进在于，每个 Transformer 层包含**两个前馈网络**，并将注意力层夹在两个前馈网络之间。可以理解为 “前馈网络 + 注意力层 + 前馈网络” 的结构，类似于一个“Macaron”三明治结构。

具体步骤如下：

1. **第一个前馈网络（FeedForward Layer 1）**：<br><br>
   - 输入首先通过一个前馈网络。这个前馈网络会对输入进行线性变换，并通过激活函数（如 ReLU 或 GELU）处理，以捕捉输入特征的非线性关系。
   - 这个前馈网络的输出会乘以 0.5，减小其影响，从而与第二个前馈网络进行平衡。
<br><br>
2. **多头自注意力机制（Multi-Head Self-Attention）**：<br><br>
   - 前馈网络的输出进入多头自注意力机制。注意力层在输入序列中建立各位置之间的依赖关系，使模型能够更好地捕捉长距离依赖。
   - 通过残差连接和 Layer Normalization，保持信息流的连续性。
<br><br>
3. **第二个前馈网络（FeedForward Layer 2）**：<br><br>
   - 注意力层的输出再通过另一个前馈网络，与第一个前馈网络具有相同的结构。这个前馈网络的输出同样乘以 0.5，与第一个前馈网络的输出形成平衡。
   - 再次通过残差连接和 Layer Normalization，得到这一层的最终输出。

- **为什么 Macaron 结构有效**？

Macaron 结构的核心是**双前馈网络**设计。相比传统 Transformer 中的单前馈网络，双前馈网络在每层中引入了更多的非线性特征转换，这带来了以下几个优势：

1. **增强非线性表达能力**：通过两个前馈网络叠加非线性变换，模型可以更好地处理复杂的特征关系，提升表示能力。
2. **更深层次的特征提取**：两个前馈网络和中间的注意力层组合，允许模型在更细粒度上对输入特征进行处理。
3. **平衡注意力和前馈网络的作用**：将两个前馈网络的输出乘以 0.5，可以让注意力层在整个层中发挥更平衡的作用，而不是完全依赖于前馈网络或注意力层。

### 3.1 多头注意力的实现与注意力可视化

<center><img src="https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2023DL/transformer/image-1.png" alt="描述文字" width="400">

在torch.nn模块下，存在**服务于Transformer架构的各类神经网络层和模型**，我们来看一下——

| 类名称                     | 作用                                          |
|--------------------------|---------------------------------------------|
| `nn.Transformer`          | 不带输入与输出层的 Transformer 模型，同时具备编码器和解码器                       |
| `nn.TransformerEncoder`   | Transformer 编码器的打包器，可以控制Nx的N的具体数字                    |
| `nn.TransformerDecoder`   | Transformer 解码器的打包器，可以控制Nx的N的具体数字                    |
| `nn.TransformerEncoderLayer` | Transformer 编码器层，由自注意力和前馈网络组成   |
| `nn.TransformerDecoderLayer` | Transformer 解码器层，由自注意力、编码器-解码器注意力和前馈网络组成 |
| `nn.MultiheadAttention`   | 多头注意力机制（注意不包含从X到QKV的转化流程）                   |
| `nn.LayerNorm`            | 层归一化                                   |
| `nn.Embedding`            | 嵌入层，用于将输入序列转换为嵌入表示          |

`torch.nn.MultiheadAttention`(embed_dim, num_heads, dropout=0.0, bias=True, add_bias_kv=False, add_zero_attn=False, kdim=None, vdim=None, batch_first=False, device=None, dtype=None)

下面是 `torch.nn.MultiheadAttention` 类中每个参数的详细作用说明：

| 参数名称                | 作用                                                                                              |
|-------------------------|---------------------------------------------------------------------------------------------------|
| `embed_dim`             | 每个注意力头输出的嵌入维度，即d_model（input_dimension）。该参数决定了输入 `query`、`key` 和 `value` 的特征维度大小。 |
| `num_heads`             | 注意力头的数量。多头注意力机制允许模型在多个子空间中并行关注不同的特征，提高模型的学习能力。               |
| `dropout`               | dropout 概率，用于在注意力权重上应用 dropout 防止过拟合。                                            |
| `bias`                  | 如果为 `True`，则在线性层中添加偏置参数。偏置项可增强模型的拟合能力。                                |
| `add_bias_kv`           | 如果为 `True`，则在 `key` 和 `value` 向量中添加一个额外的可学习偏置项，适用于特定任务。               |
| `add_zero_attn`         | 如果为 `True`，在 `key` 和 `value` 向量中添加一个全为零的向量，用于填充输入，使模型适应可变长度的输入序列。 |
| `kdim`                  | `key` 向量的特征维度。如果不设置，将默认与 `embed_dim` 相同。                                      |
| `vdim`                  | `value` 向量的特征维度。如果不设置，将默认与 `embed_dim` 相同。                                     |
| `batch_first`           | 如果为 `True`，输入的张量尺寸格式为 `(batch_size, seq_length, embed_dim)`，否则为 `(seq_length, batch_size, embed_dim)`。 |
| `device`                | 指定计算设备（如 `cuda` 或 `cpu`），默认继承模块的设备。                                               |
| `dtype`                 | 指定数据类型（如 `torch.float32`），默认继承模块的数据类型。                                            |

`nn.MultiheadAttention` 的核心是多头注意力机制，其中 `embed_dim` 和 `num_heads` 是必需参数，决定了每个注意力头的维度大小以及头的数量。其余参数是可选的，允许更细粒度地控制模型行为和性能。

MultiheadAttention的forward方法：

`forward`(query, key, value, key_padding_mask=None, need_weights=True, attn_mask=None, average_attn_weights=True, is_causal=False)

| 参数名称               | 作用                                                                                                                                                                                                                           |
|------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `query`、`key`、`value`                | Q、K、V矩阵，形状为 `(seq_len, d_model)`（未批处理时）或 `(batch_size, seq_len, d_model)`（批处理时）。  |                                                     |
| `key_padding_mask`    | 填充掩码、直接添加到相应的 `key` 值。                                                            |
| `need_weights`        | 是否返回 `attn_output_weights`，默认为 `True`。设置 `need_weights=False` 可以使用优化的 `scaled_dot_product_attention`，提升多头注意力的最佳性能。                                            |
| `attn_mask`           | 前瞻掩码、用于防止未来信息泄露的上三角掩码，作用于softmax后的矩阵 |
| `average_attn_weights`| 是否对返回的 `attn_weights` 进行头部平均。默认为 `True`，表示将注意力权重在多个头之间取平均值。当 `need_weights=True` 时有效。                                                              |
| `is_causal`           | 是否使用因果掩码作为注意力掩码。默认为 `False`。

- 有趣的need_weights

In [5]:
!pip install bertviz

   ---------------------------------------- 157.6/157.6 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 139.2/139.2 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 991.5/991.5 kB 4.5 MB/s eta 0:00:00
   ---------------------------------------- 12.7/12.7 MB 5.1 MB/s eta 0:00:00
   ---------------------------------------- 82.6/82.6 kB 2.3 MB/s eta 0:00:00


[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from transformers import BertTokenizer, BertModel
from bertviz import head_view

# 加载中文BERT模型
tokenizer = BertTokenizer.from_pretrained(r"D:\pythonwork\2024DL\model\bert-base-chinese")
model = BertModel.from_pretrained(r"D:\pythonwork\2024DL\model\bert-base-chinese", output_attentions=True)

sentence = "天下大势，合久必分，分久必合"
inputs = tokenizer(sentence, return_tensors='pt')
outputs = model(**inputs)
attention = outputs.attentions  # 获取注意力权重

tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
head_view(attention, tokens)

<IPython.core.display.Javascript object>

- 经典多头注意力的调用

In [2]:
import torch
import torch.nn as nn

# 参数定义
embedding_dim = 64   # 输入特征的维度
num_heads = 4        # 多头注意力的头数
seq_len = 10         # 序列长度
batch_size = 2       # 批次大小

# 定义多头注意力层
multihead_attn = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=num_heads)

# 创建模拟输入数据 (query, key, value)
query = torch.randn(seq_len, batch_size, embedding_dim)  # (sequence_length, batch_size, embedding_dim)
key = torch.randn(seq_len, batch_size, embedding_dim)
value = torch.randn(seq_len, batch_size, embedding_dim)

In [3]:
# 调用多头注意力
attn_output, attn_weights = multihead_attn(query, key, value)
print("Attention Output Shape:", attn_output.shape)  # 输出形状: (sequence_length, batch_size, embedding_dim)

Attention Output Shape: torch.Size([10, 2, 64])


- **layer norm**

在Transformer结构中，Layer Normalization（层归一化）是一个至关重要的部分，它是一种特定的归一化技术，与Batch Normalization（批归一化）不同，Layer Normalization不是对一个批次（batch）中的样本进行归一化，而是独立地对每个样本中的所有特征进行归一化（也就是对单一词向量、单一时间点的所有embedding维度进行归一化）。具体来说，对于每个样本，Layer Normalization会在特定层的所有激活上计算均值和方差，然后用这些统计量来归一化该样本的激活值。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/19.png)

- 基于门控前馈网络与注意力机制实现Macaron注意力层

In [18]:
import torch
import torch.nn as nn

class MLP_GLU(nn.Module):
    def __init__(self, dim: int, hidden_dim: int, dropout: float):
        super().__init__()
        
        # 定义线性层，w1 实现线性投影到 2 * hidden_dim，用于 GLU 分裂
        self.w1 = nn.Linear(dim, 2 * hidden_dim, bias=False)
        self.w2 = nn.Linear(hidden_dim, dim, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # 1. 第一个线性变换，将 x 映射到 2 * hidden_dim 维度
        h = self.w1(x)

        # 2. 将 h 分成两部分，一半作为 h 主通道，另一半作为 gate 门控
        h, gate = h.chunk(2, dim=-1)

        # 3. 使用 sigmoid 激活 gate，形成 GLU 门控
        h = h * torch.sigmoid(gate)

        # 4. 使用第二个线性层将 h 投影回 dim
        h2 = self.w2(h)

        # 5. 应用 dropout 并返回
        return self.dropout(h2)

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 定义带有前置 LayerNorm的MacaronAttention 类
class MacaronAttention(nn.Module):
    def __init__(self, dim: int, hidden_dim: int, num_heads: int, dropout: float):
        super().__init__()
        
        # 双前馈网络（GLU）
        self.mlp1 = MLP_GLU(dim, hidden_dim, dropout)
        self.mlp2 = MLP_GLU(dim, hidden_dim, dropout)
        
        # 多头注意力层的 Q、K、V 生成线性层
        # Q、K、V的结构其实与X完全一致
        self.q_proj = nn.Linear(dim, dim)
        self.k_proj = nn.Linear(dim, dim)
        self.v_proj = nn.Linear(dim, dim)
        self.attention = nn.MultiheadAttention(embed_dim=dim, num_heads=num_heads, dropout=dropout)
        
        # LayerNorm 层在每个模块之前
        self.norm1 = nn.LayerNorm(dim)
        self.norm2 = nn.LayerNorm(dim)
        self.norm3 = nn.LayerNorm(dim)
        
        # Dropout 层
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # 1. 第一个前馈网络（GLU），在 LayerNorm 前
        residual = x
        x = self.norm1(x)
        x = self.mlp1(x) * 0.5
        x = self.dropout(x) + residual

        # 2. 多头注意力层，LayerNorm 在前，QKV 单独生成
        residual = x
        x = self.norm2(x)
        q = self.q_proj(x)
        k = self.k_proj(x)
        v = self.v_proj(x)
        x, _ = self.attention(q, k, v)
        x = self.dropout(x) * 0.5 + residual

        # 3. 第二个前馈网络（GLU），在 LayerNorm 前
        residual = x
        x = self.norm3(x)
        x = self.mlp2(x) * 0.5
        x = self.dropout(x) + residual

        return x

In [20]:
dim = 64
hidden_dim = 128
num_heads = 4
dropout = 0.1
seq_len = 10
batch_size = 2

macaron_layer = MacaronAttention(dim, hidden_dim, num_heads, dropout)

# 构建示例数据
x = torch.randn(seq_len, batch_size, dim)

# 跑通Macaron层
output = macaron_layer(x)
output.shape

torch.Size([10, 2, 64])

In [21]:
output

tensor([[[-1.2400e-01,  1.3168e+00,  1.5059e+00,  ..., -3.3524e-01,
           2.1187e+00, -3.9819e-01],
         [-5.6370e-01, -2.0677e-01,  1.2180e-01,  ...,  1.6726e-01,
           1.0247e+00, -1.3022e+00]],

        [[-5.8883e-01,  1.6344e+00, -1.3060e+00,  ..., -1.7962e+00,
          -7.7542e-01, -1.3780e+00],
         [ 3.4583e-01,  3.7262e-01,  1.3585e+00,  ...,  2.5474e+00,
           5.1994e-01,  7.1703e-01]],

        [[ 4.7694e-01, -2.8462e-01,  7.3246e-01,  ..., -1.3350e+00,
           4.3372e-01,  1.2887e+00],
         [ 4.3364e-01, -1.0450e+00,  6.6667e-02,  ..., -2.9301e-01,
           4.4894e-02,  9.1819e-01]],

        ...,

        [[ 5.7591e-02,  1.3511e+00, -3.2412e-01,  ..., -2.1514e-04,
          -1.2897e+00,  1.1165e+00],
         [ 2.0748e-01, -1.1539e+00,  1.3733e+00,  ..., -3.9452e-01,
           1.0563e+00,  7.7272e-01]],

        [[ 1.9826e+00, -9.5665e-03,  1.0901e-01,  ...,  9.5151e-01,
           5.0989e-01, -5.0672e-01],
         [ 2.3985e-01,  1.0412e+0

### 3.2 reZero残差链接

**ReZero**（Residual Zero Initialization）是一种改进的残差连接方法，用于提高深层神经网络的训练稳定性。ReZero 残差链接由 Bachlechner 等人在 2020 年提出。它的核心思想是引入一个可学习的标量权重参数，使网络层的残差输出在初始时为零，并在训练过程中逐步调整这个权重。这种方式可以让残差链接变得**更稳定、更高效、更灵活、更容易收敛**。

在经典的残差连接中，网络层的输出和输入直接相加，如下所示：

$$
\text{output} = x + f(x)
$$

其中，$ x $ 是输入，$ f(x) $ 是经过当前层（如前馈网络或注意力机制）的输出。在 ReZero 中，网络层的输出会乘以一个可学习的标量参数 $ \alpha $ 后再与输入相加：

$$
\text{output} = x + \alpha \cdot f(x) \approx x
$$

并且，这个参数$\alpha$的初始值会被设置为0，构成数学上形如$x + \alpha \cdot f(x) \approx x$的效果。

相比起常规的残差链接，ReZero残差链接做出了如下改变👇

1. **初始值为零的可学习参数** $ \alpha $：
   - 在模型初始化时，$ \alpha $ 被设置为 0。这意味着在训练一开始，每层的输出都是输入 $ x $ 本身，等于跳过了这一层的非线性变换。**这种设计让模型在初始阶段保持完全的恒等映射**，降低梯度爆炸或消失的风险。
   - 具体来说，降低梯度爆炸的风险——**在深度神经网络的训练初期，层与层之间的参数还未优化，输出值可能比较随机或不稳定**。如果直接将这些初始随机输出加到输入上，可能导致层输出的变化幅度较大，增大梯度爆炸的风险。而ReZero通过初始α=0，使得这些不稳定的初始输出不会影响整体的网络输出，保证了在开始时梯度的稳定性。
   - 具体来说，降低梯度消失的风险——**ReZero的恒等映射使得梯度可以直接沿着输入 x 传播，而不会被 f(x) 的非线性函数压缩（如ReLU等）**。这就使得在模型初期，梯度可以稳定地传递到浅层。而随着训练的进行，α的值逐渐增大，每层的非线性逐步引入，不会一开始就将梯度完全压缩在初始非线性函数中。
<br><br>
2. **逐步学习的残差贡献**：
   - 在训练过程中，网络会学习逐渐增大或调整 $ \alpha $ 的值，决定当前层 $ f(x) $ 的贡献。这样，模型可以动态控制每层的输出对最终表示的影响，从而更灵活地学习不同层的特征。

---------------------------

- **原理补充：梯度大小受网络结构输出大小的影响**

模型的输出会影响梯度的大小，特别是在深层网络中，输出值的大小通过链式法则在反向传播过程中会被多次放大或缩小，这就是**梯度爆炸**和**梯度消失**的根本原因。

我们来看一个例子，设我们有一个简单的两层神经网络：
1. 输入 $ x $
2. 第一层线性变换： $ h = w_1 \cdot x $
3. 激活函数（假设是ReLU）： $ a = \text{ReLU}(h) $
4. 第二层线性变换： $ y = w_2 \cdot a $

我们定义损失函数为 $ L = \frac{1}{2} (y - t)^2 $，其中 $ t $ 是目标值。现在我们来看如何通过反向传播来计算梯度，并观察网络输出对梯度的影响——

**1. 计算损失对 $ w_2 $ 的梯度**
首先，我们对 $ w_2 $ 求导：
$$
\frac{\partial L}{\partial w_2} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial w_2}
$$
- $ \frac{\partial L}{\partial y} = y - t $
- $ \frac{\partial y}{\partial w_2} = a $

所以：
$$
\frac{\partial L}{\partial w_2} = (y - t) \cdot a
$$

**2. 计算损失对 $ w_1 $ 的梯度**

接下来，我们来看损失对 $ w_1 $ 的梯度：
$$
\frac{\partial L}{\partial w_1} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial a} \cdot \frac{\partial a}{\partial h} \cdot \frac{\partial h}{\partial w_1}
$$

逐步计算每一项：
- $ \frac{\partial L}{\partial y} = y - t $
- $ \frac{\partial y}{\partial a} = w_2 $
- $ \frac{\partial a}{\partial h} = 1 $（假设 $ h > 0 $，ReLU 导数为 1）
- $ \frac{\partial h}{\partial w_1} = x $

因此：
$$
\frac{\partial L}{\partial w_1} = (y - t) \cdot w_2 \cdot x
$$

从上面的计算中可以看到，**梯度的大小直接受到各层输出的值的影响**：
- 如果 $ y $ 或 $ w_2 $ 很大，那么 $ \frac{\partial L}{\partial w_1} $ 也会相应变大，可能导致梯度爆炸。
- 如果 $ y $ 或 $ w_2 $ 很小，接近于 0，那么 $ \frac{\partial L}{\partial w_1} $ 也会相应变小，可能导致梯度消失。

这种现象在深层网络中会被进一步放大，因为梯度会在每一层传播。如果每层的输出都很大，梯度就会在传播过程中指数级地增大，导致梯度爆炸；反之，如果每层的输出都很小，梯度会逐渐衰减，导致梯度消失。

---------------------------

- **ReZero残差链接的实现**

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 定义 MLP_GLU 类
class MLP_GLU(nn.Module):
    def __init__(self, dim: int, hidden_dim: int, dropout: float):
        super().__init__()
        self.w1 = nn.Linear(dim, 2 * hidden_dim, bias=False)
        self.w2 = nn.Linear(hidden_dim, dim, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        h = self.w1(x)
        h, gate = h.chunk(2, dim=-1)
        h = h * torch.sigmoid(gate)
        h2 = self.w2(h)
        return self.dropout(h2)

# 定义带有 ReZero 机制的 MacaronAttention 类
class MacaronAttention(nn.Module):
    def __init__(self, dim: int, hidden_dim: int, num_heads: int, dropout: float):
        super().__init__()
        
        # 双前馈网络（GLU）
        self.mlp1 = MLP_GLU(dim, hidden_dim, dropout)
        self.mlp2 = MLP_GLU(dim, hidden_dim, dropout)
        
        # 多头注意力层的 Q、K、V 生成线性层
        self.q_proj = nn.Linear(dim, dim)
        self.k_proj = nn.Linear(dim, dim)
        self.v_proj = nn.Linear(dim, dim)
        self.attention = nn.MultiheadAttention(embed_dim=dim, num_heads=num_heads, dropout=dropout)
        
        # ReZero 残差参数，初始值为 0
        self.alpha1 = nn.Parameter(torch.tensor(0.0))
        self.alpha2 = nn.Parameter(torch.tensor(0.0))
        self.alpha3 = nn.Parameter(torch.tensor(0.0))
        
        # LayerNorm 层
        self.norm1 = nn.LayerNorm(dim)
        self.norm2 = nn.LayerNorm(dim)
        self.norm3 = nn.LayerNorm(dim)
        
        # Dropout 层
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # 1. 第一个前馈网络（GLU），在 ReZero 残差连接前先进行 LayerNorm
        residual = x
        x = self.norm1(x)
        x = self.mlp1(x)
        x = residual + self.alpha1 * self.dropout(x)

        # 2. 多头注意力层，在 ReZero 残差连接前先进行 LayerNorm
        residual = x
        x = self.norm2(x)
        q = self.q_proj(x)
        k = self.k_proj(x)
        v = self.v_proj(x)
        x, _ = self.attention(q, k, v)
        x = residual + self.alpha2 * self.dropout(x)

        # 3. 第二个前馈网络（GLU），在 ReZero 残差连接前先进行 LayerNorm
        residual = x
        x = self.norm3(x)
        x = self.mlp2(x)
        x = residual + self.alpha3 * self.dropout(x)

        return x

In [23]:
# 使用示例
# 参数设置
dim = 64
hidden_dim = 128
num_heads = 4
dropout = 0.1
seq_len = 10
batch_size = 2

# 初始化 MacaronAttention 层
macaron_layer = MacaronAttention(dim, hidden_dim, num_heads, dropout)

# 生成示例输入数据
x = torch.randn(seq_len, batch_size, dim)

# 前向传播
output = macaron_layer(x)
print("Macaron Attention Output Shape:", output.shape)  # 输出形状: (seq_len, batch_size, dim)

Macaron Attention Output Shape: torch.Size([10, 2, 64])


In [24]:
output

tensor([[[ 7.2061e-01, -5.3634e-01, -4.5929e-01,  ..., -2.9354e-01,
          -5.5892e-01, -1.5828e-01],
         [-4.6246e-01,  4.2961e-01,  1.2193e+00,  ..., -1.8817e+00,
           1.3289e+00, -3.9420e-01]],

        [[-2.4468e-01,  3.7730e-01,  8.8902e-01,  ...,  9.9760e-01,
           5.9071e-01,  5.0346e-02],
         [ 4.0810e-01, -2.5330e-01,  3.6605e-01,  ...,  2.4646e-04,
           1.0346e-01, -9.7503e-01]],

        [[ 1.0957e+00,  8.3202e-01,  7.0473e-01,  ...,  6.6700e-01,
          -3.9824e-01, -8.3724e-01],
         [ 2.0771e+00, -7.6362e-01, -7.3485e-01,  ..., -9.5102e-01,
          -2.3110e-01, -5.1619e-01]],

        ...,

        [[ 1.1320e-01, -6.9277e-01,  1.0124e+00,  ..., -1.1764e+00,
          -1.7657e-01, -1.2199e+00],
         [ 7.0264e-02,  3.4717e-01,  1.0434e-01,  ...,  1.4566e-01,
           3.2979e-01, -3.5933e-01]],

        [[ 8.6193e-01,  3.3125e-01,  2.7889e+00,  ..., -5.4758e-01,
          -3.3627e-01, -2.0063e+00],
         [ 3.3083e-01,  5.8652e-0

## 4. 训练优化：梯度检查点技术

### 4.1 梯度检查点技术的原理与作用

**梯度检查点技术**（Gradient Checkpointing）是一种用于深层神经网络的内存优化方法，特别适合在计算图非常深、显存占用大的模型（如 Transformer 或 GPT 系列）中使用。梯度检查点技术通过在前向传播中存储少量的中间激活值，并在反向传播时重计算其他激活值，从而显著降低显存占用。

通常在神经网络的前向传播中，所有的中间激活值都需要存储，以便在反向传播中用于计算梯度。这种方式会消耗大量显存，尤其是对于深度网络而言。

----

**为什么要保存中间的激活值**？

存储中间的激活值（intermediate activations）在神经网络训练过程中是非常重要的，这是因为这些激活值在反向传播中用于计算梯度。在神经网络中，训练的核心步骤是**反向传播**（backpropagation），这是通过链式法则逐层计算每个参数的梯度，以便更新模型的权重。反向传播需要每层的输出激活值来计算损失函数对每个参数的偏导数。

例如，在计算每层的梯度时，下一层的梯度会依赖于当前层的激活值，这意味着每层的输出（即激活值）都需要在反向传播时用到。因此，如果在前向传播过程中不存储这些激活值，那么在反向传播时就无法计算梯度。

反向传播使用的是链式法则，即：

$$
\frac{\partial L}{\partial x} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial x}
$$

在每一层，计算损失 $L$ 对于某一参数的导数（如权重和偏置）时，链式法则需要当前层的输出值作为下一层输入的一部分。因此，网络的每一层都需要知道前一层的输出，以计算出正确的梯度。

----

梯度检查点技术通过以下方法来减少显存的消耗：

1. **前向传播时设置检查点**：<br>
   - 在前向传播过程中，我们将网络分成多个“检查点”（checkpoints）。每个检查点保存当前层的激活值，但不保存该检查点之后的所有中间激活值。
   - 比如，如果有一个深层网络，我们可以每隔几层（例如每5层）设置一个检查点。这样，除了检查点，我们只保存少量的激活值，其他中间层的激活值都不保存。
<br><br>
2. **反向传播时的激活值重计算**：
   - 在反向传播过程中，当我们需要某一层的激活值来计算梯度时，如果该层的激活值没有存储，我们从最近的检查点重新计算这部分网络的前向传播，直到计算到当前层的激活值。
   - 这样做的好处是，尽管我们增加了计算量（因为我们重新计算了一部分前向传播），但显存占用量减少了，因为不需要在前向传播中存储所有激活值。
<br><br>
3. **显存与计算的权衡**：
   - 由于反向传播时需要重算激活值，因此增加了计算量。然而，显存占用得到了显著降低，因此梯度检查点技术是一种**用计算量换显存的策略**，特别适用于显存受限但计算资源充足的环境。

----

**具体步骤**

假设我们有一个简单的网络，包含10层，激活值依次记为 $ a_1, a_2, \dots, a_{10} $，并在第1层和第5层设置检查点。

1. **前向传播**：
   - 在前向传播过程中，只有第1层和第5层的激活值 $ a_1 $ 和 $ a_5 $ 被存储，其他层的激活值（例如 $ a_2, a_3, \dots, a_4 $）不存储。
   <br><br>
2. **反向传播**：
   - 当需要计算第10层的梯度时，反向传播从第10层依次往回传播，但会依赖于第9层的激活值。
   - 如果第9层的激活值未被存储，系统会从第5层的检查点重新执行前向传播，重新计算出 $ a_6, a_7, a_8, a_9 $ 的激活值。
   - 继续反向传播时，依次对第8层、第7层等进行梯度计算，直到处理完所有层。

通过这种方式，**只需存储少量的检查点激活值**，在反向传播时动态地重算其他激活值。尽管这增加了计算量，但显存占用大幅减少，使得可以在显存受限的情况下训练更深的网络。

### 4.2 大模型显卡适配与选择指南

**显卡计算资源由什么决定？计算量换显存可行吗？**

**显存**主要决定了显卡能够处理的数据量，比如加载模型和数据的大小，而**计算能力**则决定了显卡处理数据的速度和效率。

对于计算能力来说，以下三个指标确实是最核心的评估维度：

1. **CUDA 核心数量**：
   - CUDA 核心是 GPU 的基础计算单元，负责执行基本的并行计算任务。**CUDA 核心数量越多，显卡的并行计算能力越强，理论上计算能力越高**。
   - 不过，CUDA 核心数量只是一个粗略的指标，不同架构的 CUDA 核心之间的性能差异可能较大。例如，Ampere 架构的 CUDA 核心比上一代 Turing 架构效率更高。
<br><br>
2. **Tensor Cores**：
   - **Tensor Cores 是专为深度学习设计的计算单元，可以加速矩阵运算**，尤其是混合精度（FP16 和 FP32）下的矩阵乘法运算。Tensor Cores 的引入显著提升了深度学习任务中的计算速度，尤其是大规模的神经网络训练。
   - 例如，使用 Tensor Cores 的 A100 在 FP16 运算上比只用 CUDA 核心的 GPU 快得多。
<br><br>
3. **FLOPS（每秒浮点运算次数）**：
   - FLOPS 是衡量 GPU 理论计算能力的一个核心指标。一般来说，FP32、FP16 和 INT8 的 FLOPS 值会分别影响不同精度下的计算效率。**FLOPS 值越高，表示显卡在该精度下的理论计算速度越快**。
   - 不过，实际任务中的性能还会受到其他因素（如显存带宽和延迟）的影响，FLOPS 只是计算能力的一个理论上限。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/21.png)

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/22.png)

| 显卡型号      | 架构   | CUDA 核心 | Tensor Cores | 显存   | 显存带宽  | FP32 性能 | 适用场景                   |
|--------------|--------|-----------|--------------|--------|-----------|-----------|----------------------------|
| A100         | Ampere | 6912      | 432          | 40/80GB HBM2e | 1555/2039 GB/s | 19.5 TFLOPS | 深度学习训练、大规模计算        |
| H100         | Hopper | 14592     | 528          | 80GB HBM3     | 3 TB/s    | 60 TFLOPS   | 超大规模模型训练             |
| A6000        | Ampere | 10752     | 336          | 48GB GDDR6    | 768 GB/s  | 38.7 TFLOPS | 图形设计、AI 加速            |
| A40          | Ampere | 10240     | 320          | 48GB GDDR6    | 696 GB/s  | 37.4 TFLOPS | AI 推理、虚拟化工作站         |
| T4           | Turing | 2560      | 320          | 16GB GDDR6    | 320 GB/s  | 8.1 TFLOPS  | 中小规模 AI 推理，云计算       |
| V100         | Volta  | 5120      | 640          | 16/32GB HBM2  | 900 GB/s  | 15.7 TFLOPS | 深度学习训练、HPC          |
| RTX A5000    | Ampere | 8192      | 256          | 24GB GDDR6    | 600 GB/s  | 27.8 TFLOPS | 图形工作站、AI 加速          |

- **A100 和 H100**：面向深度学习训练和大规模科学计算，尤其是 H100，适合超大规模的模型训练。
- **A6000 和 A40**：用于图形工作站和 AI 加速应用，适合需要较大显

存和图形处理能力的任务。
- **T4 和 V100**：性价比高的选择，适合中小规模推理和云计算环境。
- **RTX A5000**：适合工作站用户，图形处理和 AI 加速能力均衡。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2024LLM/20.png)

**优点与劣势**——

1. **减少显存占用**：通过仅存储部分激活值，梯度检查点技术可以显著减少深度神经网络的显存占用，使得在有限显存的情况下也能训练超大模型。
2. **更大批量训练**：节省的显存可以用于增加批量大小，进而提高训练效率和模型性能。
3. **扩展更深层网络的可训练性**：对于深度网络，传统方法的显存占用可能超过硬件限制，而梯度检查点技术可以使得更深层的模型在现有硬件上训练成为可能。
4. **增加计算量**：在反向传播中需要重新计算未存储的激活值，导致计算量增加，训练速度变慢。
5. **实现复杂性**：实现梯度检查点技术需要对计算图进行分段和合理的检查点设置，稍微增加了代码的复杂性。

**梯度检查点技术适合以下情况**：

- 超大规模模型：如 GPT-3 或 BERT 等具有上百层的深度网络。
- 显存有限的硬件环境：如在单块 GPU 或中等显存资源的集群上训练超大模型。
- 需要大批量训练：在显存空间紧张的情况下，为了提高训练速度或模型效果，使用梯度检查点可以腾出显存来增加批量大小。

**在 PyTorch 中的实现**

PyTorch 提供了 `torch.utils.checkpoint` 模块，可以方便地实现梯度检查点技术。使用时，只需将计算图中的某些部分用 `checkpoint` 包装起来即可。例如：

```python
import torch
from torch.utils.checkpoint import checkpoint

class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layer1 = torch.nn.Linear(1024, 1024)
        self.layer2 = torch.nn.Linear(1024, 1024)
        self.layer3 = torch.nn.Linear(1024, 1024)

    def forward(self, x):
        x = checkpoint(self.layer1, x)  # 设置检查点，不进行存储
        x = checkpoint(self.layer2, x)  # 设置检查点，不进行存储
        x = self.layer3(x)  # 最后一层不需要检查点，也就是默认状态，进行储存
        return x

# 使用模型
model = MyModel()
input_tensor = torch.randn(16, 1024)
output = model(input_tensor)
```

在这个示例中，`layer1` 和 `layer2` 的激活值会在反向传播中被重计算，而不是存储在显存中。这种方式减少了显存使用，同时保留了反向传播的准确性。

### 4.3 梯度检查点的实现以及对内存的影响

在PyTorch中，**`torch.utils.checkpoint`** 提供了梯度检查点的工具，可以方便地在网络中应用梯度检查点技术。**你在哪一个层、或者哪一个元素上使用checkpoint，就可以使那个元素上的激活值进入不保存状态**。使用方式如下：

以下是 `torch.cuda` 常用函数及其作用和使用场景的表格版本：

| **函数名**                        | **作用**                                                                                         | **使用场景**                                                                                     | **备注**                                      |
|-----------------------------------|--------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------|-----------------------------------------------|
| `torch.cuda.memory_allocated()`   | 返回当前 GPU 上实际分配的显存量（字节）。                                                        | 检查模型和数据占用的显存量。                                                                     | 不包括缓存显存，仅实际分配部分。              |
| `torch.cuda.memory_reserved()`    | 返回 GPU 上 PyTorch 预分配的显存量，包括缓存部分。                                                | 查看 PyTorch 总的显存预分配情况。                                                                | 包括 `memory_allocated` 和缓存的显存。        |
| `torch.cuda.max_memory_allocated()`| 返回程序运行过程中最大显存分配量。                                                               | 测试代码或模型的显存分配峰值。                                                                   | 通常与 `reset_peak_memory_stats()` 配合使用。 |
| `torch.cuda.max_memory_reserved()`| 返回程序运行过程中最大显存预分配量。                                                             | 测试显存预分配的峰值情况。                                                                       | 同样需要重置统计信息。                        |
| `torch.cuda.empty_cache()`        | 清空 PyTorch 的显存缓存，将未使用的显存释放回 CUDA。                                              | 避免缓存显存过多引发的 OOM 错误。                                                                | 实际分配的显存不会被释放。                    |
| `torch.cuda.reset_peak_memory_stats()` | 重置显存分配的峰值统计信息。                                                                   | 测量某段代码的显存峰值使用情况。                                                                 | 在测量显存前调用。                            |
| `torch.cuda.is_available()`       | 检查是否支持 CUDA。                                                                               | 在运行模型前检测是否有可用的 GPU。                                                               | 用于兼容 GPU 和 CPU 的代码。                  |
| `torch.cuda.device_count()`       | 返回当前可用 GPU 的数量。                                                                        | 检查多 GPU 环境的设备数量。                                                                      |                                                |
| `torch.cuda.current_device()`     | 返回当前默认使用的 GPU 的编号。                                                                  | 检查当前运行代码使用的是哪个 GPU。                                                               |                                                |
| `torch.cuda.set_device(device)`   | 设置默认使用的 GPU。                                                                             | 在多 GPU 环境中，切换到指定 GPU 运行代码。                                                       | `device` 可以是 GPU 编号或 `torch.device`。   |
| `torch.cuda.device(device)`       | 上下文管理器，在指定 GPU 上运行代码。                                                            | 临时切换 GPU 执行代码。                                                                          | 适合临时切换，不改变全局默认 GPU。            |
| `torch.cuda.synchronize()`        | 强制同步 GPU 上的所有操作，确保计算完成。                                                        | 测量运行时间或显存占用时，确保结果准确。                                                         | GPU 操作是异步的，未同步可能导致测量偏差。    |
| `torch.cuda.stream()`             | 创建并管理 CUDA 流（streams）。                                                                  | 在不同流中并行运行多个任务以提高 GPU 计算效率。                                                  |                                                |
| `torch.cuda.memory_stats()`       | 返回 GPU 显存使用的详细统计信息，包括分配次数、未释放的内存块等。                                | 调试显存分配，排查 OOM 等问题。                                                                  |                                                |
| `torch.cuda.profiler`             | 提供 CUDA 性能分析工具。                                                                         | 分析模型的性能瓶颈，优化 GPU 计算效率。                                                          |                                                |

---

- **通常使用状况**
1. **显存监控**：优先使用 `memory_allocated` 和 `max_memory_allocated` 监测模型显存占用。
2. **设备管理**：多 GPU 环境中，配合 `set_device` 和 `device_count`。
3. **性能调试**：结合 `profiler` 和 `stream` 进行优化

In [41]:
import torch
from torch import nn
from torch.utils.checkpoint import checkpoint

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layer1 = nn.Linear(512, 512)
        self.layer2 = nn.Linear(512, 512)
        self.layer3 = nn.Linear(512, 512)

    def forward(self, x, use_checkpoint = False):
        if use_checkpoint:
            # 设置检查点
            x = checkpoint(self.layer1, x)
            x = self.layer2(x)
            x = checkpoint(self.layer3, x)
        else:
            x = self.layer1(x)
            x = self.layer2(x)
            x = self.layer3(x)
        return x

# 创建模型和输入
model = MyModel().cuda()
input_data = torch.randn(1000, 512).cuda()

In [42]:
# 显存测量函数
def measure_memory(model, input_data, use_checkpoint=False):
    torch.cuda.empty_cache()  # 清空显存缓存
    torch.cuda.reset_peak_memory_stats()  # 重置显存统计

    torch.cuda.synchronize()  # 同步 GPU 计算，帮助torch.cuda的监控结果与nvidia-smi保持一致
    output = model(input_data, use_checkpoint=use_checkpoint)
    loss = output.sum()
    loss.backward()
    torch.cuda.synchronize()  # 确保反向传播完成

    # 获取内存分配情况
    allocated = torch.cuda.max_memory_allocated() / 1024**2  # MB
    reserved = torch.cuda.max_memory_reserved() / 1024**2  # MB
    return allocated, reserved

In [43]:
# 测量未使用梯度检查点的显存
allocated_no_checkpoint, reserved_no_checkpoint = measure_memory(model, input_data, use_checkpoint=False)
print(f"[No Checkpoint] Allocated: {allocated_no_checkpoint:.2f} MB, Reserved: {reserved_no_checkpoint:.2f} MB")

[No Checkpoint] Allocated: 60.90 MB, Reserved: 72.00 MB


In [44]:
# 测量使用梯度检查点的显存
allocated_with_checkpoint, reserved_with_checkpoint = measure_memory(model, input_data, use_checkpoint=True)
print(f"[With Checkpoint] Allocated: {allocated_with_checkpoint:.2f} MB, Reserved: {reserved_with_checkpoint:.2f} MB")

[With Checkpoint] Allocated: 65.86 MB, Reserved: 74.00 MB


In [45]:
# 显存节约对比
allocated_saving = (allocated_no_checkpoint - allocated_with_checkpoint) / allocated_no_checkpoint * 100
print(f"Memory saving (allocated): {allocated_saving:.2f}%")

Memory saving (allocated): -8.14%


使用checkpoint之后，可以看到所需的内存减少了8%左右，当应用于复杂的网络时省的内存会更多。

- **假设把所有的层都设上梯度检查点呢**？

In [37]:
import torch
from torch import nn
from torch.utils.checkpoint import checkpoint

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layer1 = nn.Linear(512, 512)
        self.layer2 = nn.Linear(512, 512)
        self.layer3 = nn.Linear(512, 512)

    def forward(self, x, use_checkpoint = False):
        if use_checkpoint:
            # 设置检查点
            x = checkpoint(self.layer1, x)
            x = checkpoint(self.layer2, x)
            x = checkpoint(self.layer3, x)
        else:
            x = self.layer1(x)
            x = self.layer2(x)
            x = self.layer3(x)
        return x

# 创建模型和输入
model = MyModel().cuda()
input_data = torch.randn(1000, 512).cuda()

如你所见，正向传播依然可以运行，然而——

In [38]:
# 测量未使用梯度检查点的显存
allocated_no_checkpoint, reserved_no_checkpoint = measure_memory(model, input_data, use_checkpoint=False)
print(f"[No Checkpoint] Allocated: {allocated_no_checkpoint:.2f} MB, Reserved: {reserved_no_checkpoint:.2f} MB")

[No Checkpoint] Allocated: 70.82 MB, Reserved: 78.00 MB


In [40]:
# 测量使用梯度检查点的显存
allocated_with_checkpoint, reserved_with_checkpoint = measure_memory(model, input_data, use_checkpoint=True)
print(f"[With Checkpoint] Allocated: {allocated_with_checkpoint:.2f} MB, Reserved: {reserved_with_checkpoint:.2f} MB")

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

从报错信息 "RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn" 来看，这个问题是因为所有层都使用了 checkpoint，导致整个计算图的梯度信息丢失（即没有 grad_fn），从而无法完成反向传播。

这个问题的根本原因是：在梯度检查点技术中，至少需要有一部分计算图的节点（激活值）被保存下来，否则计算图无法正确建立，无法反向传播。

- **GPT的梯度检查点的实现**

对于Transformer、GPT等堆叠重复模块的架构，梯度检查点的最佳设置通常是 **在重复模块之间的边界**，而不是在一个模块的内部。这意味着：

1. **按模块级别设置检查点**：
   - 每个解码器模块（Decoder Block）可以作为一个独立单元。
   - 假设堆叠 24 层解码器模块，可以每隔 2-4 层设置一个检查点。例如：
   - 设置检查点在第4层、第8层、第12层……这样在重算时，每次只需重算 4 层，而不需要重算整个模型。
<br><br>
2. **跳过固定的输入和输出部分**：
   - 输入嵌入层和输出层通常计算量较低，但显存占用不多，因此这些部分的激活值可以直接存储，无需检查点。
   - 重点优化的是解码器模块，因为这些模块不仅堆叠多次，还包含了显存占用较大的部分（如注意力和前馈网络的激活值）。
<br><br>
3. **跨 Encoder 和 Decoder 的共享部分**（适用于完整的 Transformer 架构）：
   - 如果是完整的 Transformer（即 Encoder + Decoder），Encoder 和 Decoder 是独立的模块，可以单独设置检查点。
   - Encoder 的检查点可以每隔几层分配，Decoder 也同样如此。

后续当我们实现了GPT所有的优化技术之后、我们将会再来检查梯度检查点带来的内存缩减的影响。

## 5. 推理加速：基于KV缓存的增量推理技术

推理增量技术和梯度检查点技术是GPT模型训练和推理中非常关键的优化手段，主要用于减少内存占用和提高计算效率。首先，
**推理增量**（Incremental Inference / Incremental Decoding）是一种通过逐步生成序列的方式来优化内存和计算开销的技术。在GPT模型中，推理增量技术的核心是KV缓存（Key-Value Cache）。这种缓存机制存储了每一步生成的特征，避免每次都从头计算，使生成序列变得更高效。

在GPT模型中，KV缓存的设计是为了支持**逐步生成**的自回归推理。在生成序列时，GPT会逐步将每一步的输出缓存下来，以便后续推理时可以减少重复计算。其实现的特点包括：

- **全局缓存设计**：GPT的KV缓存一般在生成序列时将所有注意力层的KV值都缓存下来，以便在下一步生成时能直接访问。这样，每生成一个新单词时，模型仅需更新当前步的查询（Q）向量。
- **缓存管理**：缓存的Key和Value在整个推理过程中保留到结束，不会进行重置或清空，这有助于提高生成长文本时的计算效率。
- **应用场景**：这种缓存主要用于生成任务的增量推理，不适用于非自回归的任务。

LLaMA也使用KV缓存，但在一些细节上有所优化，尤其是面向更大的模型和多样化的生成需求。以下是LLaMA的KV缓存机制中的一些特色：

- **部分头的KV缓存**：LLaMA的实现中，可能并非所有注意力头都使用KV缓存（取决于具体实现），这使得它能灵活控制缓存的粒度。例如，部分注意力头会在推理过程中动态启用或关闭KV缓存，这种机制可以减少部分场景下的缓存开销。
- **更细粒度的KV管理**：LLaMA的KV缓存管理更灵活。例如，对于较长生成序列，LLaMA会考虑清理或重置部分不重要的KV值，以优化缓存使用。
- **针对长序列的优化**：LLaMA在长文本生成任务中，采取了更高效的KV缓存策略，以支持更长的序列生成。这意味着其缓存实现可能会更注重效率，确保生成超长文本时的缓存利用率。

总之，GPT和LLaMA的KV缓存的本质是相同的，都是通过缓存Key和Value来减少计算量、加速推理，但LLaMA的实现更具**灵活性**，尤其在支持更长序列生成和更细粒度缓存控制方面有所改进。这些差异主要体现在特定的实现细节和应用场景的适应性上。

## 6. 从0实现GPT3架构

In [75]:
import math
import torch
from torch import nn
import torch.nn.functional as F
from typing import Any, Optional, Tuple
from transformers import PretrainedConfig
from transformers import PreTrainedModel
import torch.utils.checkpoint as checkpoint
from transformers.modeling_outputs import CausalLMOutputWithPast

class LMConfig(PretrainedConfig):
    model_type = "MateConv_GPT"
    
    def __init__(
        self,
        vocab_size=6400,
        max_seq_len=1024,
        dim=768,
        hidden_dim=3072,
        n_heads=12,
        n_layers=12,
        dropout=0.1,
        norm_eps=1e-5,
        use_checkpoint=False  # 是否使用梯度检查点
    ):
        """
        GPT 模型的配置类，定义所有模型参数。
        """
        self.vocab_size = vocab_size  # 词汇表大小
        self.max_seq_len = max_seq_len  # 最大序列长度
        self.dim = dim  # 嵌入维度
        self.hidden_dim = hidden_dim  # 前馈网络隐藏层维度
        self.n_heads = n_heads  # 注意力头数量
        self.n_layers = n_layers  # TransformerBlock 的层数
        self.dropout = dropout  # Dropout 概率
        self.norm_eps = norm_eps  # LayerNorm 的 epsilon
        self.use_checkpoint = use_checkpoint  # 是否使用梯度检查点

        # 计算每个注意力头的维度
        if dim % n_heads != 0:
            raise ValueError("`dim` must be divisible by `n_heads`.")
        self.head_dim = dim // n_heads

In [87]:
# 轴向位置编码
def axial_positional_emb(embedding_dim, axial_dim_1, axial_dim_2):
    axial_wpe_1 = nn.Parameter(torch.randn(axial_dim_1, embedding_dim) * 0.01)
    axial_wpe_2 = nn.Parameter(torch.randn(axial_dim_2, embedding_dim) * 0.01)

    # 广播并组合轴向位置编码
    axial_wpe_1 = axial_wpe_1.unsqueeze(1).expand(-1, axial_dim_2, embedding_dim)
    axial_wpe_2 = axial_wpe_2.unsqueeze(0).expand(axial_dim_1, -1, embedding_dim)
    wpe = (axial_wpe_1 + axial_wpe_2) / 2

    return wpe.view(axial_dim_1 * axial_dim_2, embedding_dim)

class MLP_GLU(nn.Module):
    def __init__(self, dim: int, hidden_dim: int, dropout: float):
        super().__init__()
        self.w1 = nn.Linear(dim, 2 * hidden_dim, bias=False)
        self.w2 = nn.Linear(hidden_dim, dim, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        h = self.w1(x)
        h, gate = h.chunk(2, dim=-1)
        h = h * torch.sigmoid(gate)
        h2 = self.w2(h)
        return self.dropout(h2)

# Transformer Block（基于 MacaronAttention）
class TransformerBlock(nn.Module):
    def __init__(self, layer_id: int, config: LMConfig):
        super().__init__()
        self.layer_id = layer_id

        # 自注意力
        self.attention = nn.MultiheadAttention(embed_dim=config.dim, num_heads=config.n_heads, dropout=config.dropout)

        # GLU 的双前馈层
        self.ffn1 = MLP_GLU(config.dim, config.hidden_dim, config.dropout)
        self.ffn2 = MLP_GLU(config.dim, config.hidden_dim, config.dropout)

        # ReZero 参数
        self.alpha1 = nn.Parameter(torch.tensor(0.0))
        self.alpha2 = nn.Parameter(torch.tensor(0.0))
        self.alpha3 = nn.Parameter(torch.tensor(0.0))

        # LayerNorm 和 Dropout
        self.norm1 = nn.LayerNorm(config.dim, eps=config.norm_eps)
        self.norm2 = nn.LayerNorm(config.dim, eps=config.norm_eps)
        self.norm3 = nn.LayerNorm(config.dim, eps=config.norm_eps)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x, pos_cis):
        # 前馈网络 1
        residual = x
        x = self.norm1(x)
        x = self.ffn1(x)
        x = residual + self.alpha1 * self.dropout(x)

        # 多头注意力
        residual = x
        x = self.norm2(x)
        x, _ = self.attention(x, x, x)
        x = residual + self.alpha2 * self.dropout(x)

        # 前馈网络 2
        residual = x
        x = self.norm3(x)
        x = self.ffn2(x)
        x = residual + self.alpha3 * self.dropout(x)

        return x

# GPT 模型
class GPT(PreTrainedModel):
    config_class = LMConfig  # 定义模型使用的配置类
    last_loss: Optional[torch.Tensor]  # 用于记录最后计算的损失值
    
    def __init__(self, params: LMConfig = None):
        """
        GPT 是一个基于 Transformer 架构的语言模型，继承自 PreTrainedModel。

        参数：
        - params: 配置对象 LMConfig，包含模型的超参数配置。
        """
        super().__init__(params)
        if not params:
            params = LMConfig()  # 如果没有提供配置，则使用默认配置
        self.params = params  # 保存模型参数配置
        self.vocab_size = params.vocab_size  # 词汇表大小
        self.n_layers = params.n_layers  # Transformer 的层数

        # 词嵌入和轴向位置编码
        self.embedding = nn.Embedding(config.vocab_size, config.dim)
        self.dropout = nn.Dropout(config.dropout)
        self.pos_emb = axial_positional_emb(config.dim, config.max_seq_len, config.max_seq_len)

        # Transformer 层
        self.blocks = nn.ModuleList([TransformerBlock(i, config) for i in range(config.n_layers)])
        self.norm = nn.LayerNorm(config.dim, eps=config.norm_eps)

        # 输出层：共享嵌入权重
        self.output = nn.Linear(config.dim, config.vocab_size, bias=False)
        self.output.weight = self.embedding.weight  # 共享词嵌入层和输出层的权重

        #初始化模型权重
        self.apply(self._init_weights)  # 初始化模型权重
        self.OUT = CausalLMOutputWithPast()  # 初始化OUT类
        #OUT类用于封装模型的输出信息，它不参与模型的计算，而是帮助管理和返回输出结果

    def _init_weights(self, module):
        """
        初始化模块权重。
        - 对线性层使用正态分布初始化权重，均值为 0，标准差为 0.02。
        - 对词嵌入层也使用正态分布初始化权重，均值为 0，标准差为 0.02。
        """
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, tokens: Optional[torch.Tensor] = None
                , targets: Optional[torch.Tensor] = None
                , **keyargs):
        """
        GPT 的前向传播函数。

        参数：
        - tokens: 输入的 token 张量，表示输入的词序列。
        - targets: 目标张量，用于计算交叉熵损失。
        - kv_cache: 是否使用键值缓存（用于加速推理）。
        - keyargs: 其他可选参数，如 'input_ids' 和 'attention_mask'。

        返回：
        - 输出的 logits 和 loss（如果有目标）。
        """
        current_idx = 0  # 当前索引初始化为 0
        if 'input_ids' in keyargs:
            tokens = keyargs['input_ids']  # 从关键字参数中提取 'input_ids'
        if 'attention_mask' in keyargs:
            targets = keyargs['attention_mask']  # 从关键字参数中提取 'attention_mask'
        if 'current_idx' in keyargs:
            current_idx = int(keyargs['current_idx'])  # 更新当前索引

        # 获取输入 tokens 的序列长度
        seq_len = tokens.size(1)

        # 嵌入层 + 位置编码 + dropout
        h = self.embedding(tokens) + self.pos_emb[:seq_len, :].to(tokens.device)
        h = self.dropout(h)

        # 梯度检查点技术
        for idx, block in enumerate(self.blocks):
            if self.params.use_checkpoint and idx % 4 != 0:  # 使用梯度检查点，每 4 个保存一次激活
                h = checkpoint.checkpoint(block, h, self.pos_emb.to(tokens.device))
            else:  # 完整保存激活值
                h = block(h, self.pos_emb.to(tokens.device))
        
        # 最后的layer norm
        h = self.norm(h)
        
        if targets is not None:
            logits = self.output(h)  # 通过线性输出层生成 logits
            # 计算交叉熵损失，忽略 index 为 0 的位置，reduction 为 'none'，即不自动求平均
            self.last_loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1),
                                             ignore_index=0, reduction='none')
        else:
            logits = self.output(h[:, [-1], :])  # 如果没有目标，只返回最后一个时间步的 logits
            self.last_loss = None  # 没有损失
        
        self.OUT.__setitem__('logits', logits)  # 设置输出的 logits
        self.OUT.__setitem__('last_loss', self.last_loss)  # 设置最后的 loss
        return self.OUT  # 返回输出对象

    @torch.inference_mode()
    def generate(self, idx, eos, max_new_tokens, temperature=0.7, top_k=8, stream=True, rp=1., kv_cache=True):
        """
        推理模式下的文本生成函数。

        参数：
        - idx: 输入的 tokens。
        - eos: 结束标志符号，当生成到 eos 时停止生成。
        - max_new_tokens: 最大生成的新 token 数量。
        - temperature: 控制生成的随机性，温度越高，生成越多样化。
        - top_k: 限制 top-k 采样，控制只选择概率最高的 k 个 token。
        - stream: 是否进行流式输出。
        - rp: 重复惩罚系数，控制重复 token 的惩罚。
        - kv_cache: 是否使用键值缓存来加速推理。

        返回：
        - 生成的 tokens（可能是流式返回）。
        """
        index = idx.shape[1]  # 获取输入 token 序列的长度
        init_inference = True  # 初始化推理标志
        while idx.shape[1] < max_new_tokens - 1:  # 当生成的 tokens 长度小于最大 tokens 数时继续生成
            if init_inference or not kv_cache:
                inference_res, init_inference = self(idx, kv_cache=kv_cache), False  # 第一次推理，或不使用缓存
            else:
                inference_res = self(idx[:, -1:], kv_cache=kv_cache, current_idx=idx.shape[1] - 1)  # 仅使用最后一个 token 推理

            logits = inference_res.logits  # 获取推理结果的 logits
            logits = logits[:, -1, :]  # 只选择最后一个 token 的 logits

            # 对生成的 token 进行重复惩罚
            for token in set(idx.tolist()[0]):
                logits[:, token] /= rp  # 对每个重复的 token 施加惩罚

            if temperature == 0.0:  # 如果温度为 0，使用贪心算法选择下一个 token
                _, idx_next = torch.topk(logits, k=1, dim=-1)
            else:
                logits = logits / temperature  # 根据温度调整 logits
                if top_k is not None:
                    v, _ = torch.topk(logits, min(top_k, logits.size(-1)))  # 使用 top-k 采样
                    logits[logits < v[:, [-1]]] = -float('Inf')  # 排除 top-k 之外的 logits

                probs = F.softmax(logits, dim=-1)  # 计算概率分布
                idx_next = torch.multinomial(probs, num_samples=1, generator=None)  # 根据概率进行采样

            if idx_next == eos:  # 如果生成了 eos token，停止生成
                break

            idx = torch.cat((idx, idx_next), dim=1)  # 将生成的 token 拼接到输入序列中
            if stream:  # 如果启用了流式输出
                yield idx[:, index:]  # 输出当前生成的 tokens

        if not stream:  # 如果未启用流式输出
            yield idx[:, index:]  # 返回生成的完整序列

In [88]:
config = LMConfig()
model = GPT(config)

# 测试输入
tokens = torch.randint(0, config.vocab_size, (1, 10))  # 模拟一个 batch size 为 1 的 10 个 token 输入
targets = torch.randint(0, config.vocab_size, (1, 10))

# 测试前向传播
output = model(tokens=tokens, targets=targets)
print("Logits:", output['logits'])
print("Loss:", output['last_loss'])

# 测试生成
print("\nGenerated Sequence:")
eos_token = 50256  # 假设 50256 是结束符
for generated in model.generate(tokens, eos=eos_token, top_k = 30
                                , max_new_tokens=20, temperature=0.7):
    print(generated)

Logits: tensor([[[ 0.4740,  0.4596,  0.2333,  ...,  0.6799, -0.0197, -0.3567],
         [ 0.1748, -0.3398,  0.0329,  ..., -0.1562,  0.1060,  0.2435],
         [ 0.1187,  0.5350, -0.7705,  ...,  0.6233, -0.3082, -0.1057],
         ...,
         [-0.2179, -0.0146,  0.8750,  ..., -0.6568, -0.2955,  0.4934],
         [-0.3457, -0.2445,  0.0478,  ...,  0.1752,  0.9795,  0.0749],
         [-0.0205,  0.8334,  0.9906,  ..., -0.1893, -0.1394, -0.0656]]],
       grad_fn=<UnsafeViewBackward0>)
Loss: tensor([12.8729, 12.8166, 14.2058, 14.2633, 12.5228, 13.1014, 14.8954, 13.3678,
        13.8373, 12.5061], grad_fn=<NllLossBackward0>)

Generated Sequence:
tensor([[2601]])
tensor([[2601, 2601]])
tensor([[2601, 2601, 2601]])
tensor([[2601, 2601, 2601, 2601]])
tensor([[2601, 2601, 2601, 2601, 2601]])
tensor([[2601, 2601, 2601, 2601, 2601, 2601]])
tensor([[2601, 2601, 2601, 2601, 2601, 2601, 2601]])
tensor([[2601, 2601, 2601, 2601, 2601, 2601, 2601, 2601]])
tensor([[2601, 2601, 2601, 2601, 2601, 2601, 2